This is the first notebook of the pipeline. The project is structured around three folders: Acquisition, Analysis and Code. 
Acquisition stores all the data, Analysis the results (and the intermediary results) and Code, the present notebooks. 
Both in Acquisition and Analysis we create one folder per experiment type (called batch). Inside each batch folder, we will keep a similar folder structure.

Each batch will be stored in separate folder inside the analysis folder in which all the results will be stored.

 Jacques Bourg @ Florian Muller lab. Institut Pasteur.



<div style="background-color: white; padding: 10px;">
    <img src="./pipeline.png" alt="pipeline" width="1200" height="420">
</div>

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import skimage.io as io
import shutil

import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../pipeline/src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))

from utils.parameters_tracking import Parameter_tracking as Track
from utils.widgets import StringListWidget as Slw
from utils.file_handling import FileProcessor as Fp

fp  = Fp()
tk  = Track()

Ensure folders Acquisition and Analysis exist, otherwise create them.

In [ ]:
folder_analysis_batch = Path(f"../Acquisition")
if not folder_analysis_batch.exists():
    folder_analysis_batch.mkdir(parents=True)
    
folder_analysis_batch = Path(f"../Analysis")
if not folder_analysis_batch.exists():
    folder_analysis_batch.mkdir(parents=True) 

In [ ]:
text_input = widgets.Text(value='',placeholder='Batch name to input', description='', disabled=False)
display(text_input)

In [ ]:
batch_name = text_input.value; print(batch_name)

In [ ]:
folder_path_batch = Path(f"../Analysis/{batch_name}")
if not folder_path_batch.exists():
    folder_path_batch.mkdir(parents=True)
    
folder_path_batch = Path(f"../Acquisition/{batch_name}")
if not folder_path_batch.exists():
    folder_path_batch.mkdir(parents=True)

This line defines three types of experiments/conditions that will take place in this batch: LNP, XPO and EP (electroporation). We will create a single folder for each condition.
Set the names in capitals ! 

In [ ]:
mods = Slw(); mods.display()

In [ ]:
modalities = mods.string_list; print(modalities)

In [ ]:
for mod in modalities:
    folder_path_modality = Path(f"../Analysis/{batch_name}/{mod}")
    if not folder_path_modality.exists():
        folder_path_modality.mkdir(parents=True)

for mod in modalities:
    folder_path_modality = Path(f"../Acquisition/{batch_name}/{mod}")
    if not folder_path_modality.exists():
        folder_path_modality.mkdir(parents=True)

Now, lets defines the types of channels that appear in this experiment, for instance: DAPI and FISH (if there are several fish call them FISH_1, FISH_2 or some other name).
Set the names in capitals !

In [ ]:
chans = Slw(); chans.display()

In [ ]:
channels = chans.string_list; print(channels)

Associate a color with each channel, ni the same order (write in lowercase or in hex), this will be useful when plotting the results

In [ ]:
cols = Slw(); cols.display()

In [ ]:
colors = cols.string_list; print(colors)

Enter the two structures we are going to segment (usually NUCLEI and CELL in uppercase)

In [ ]:
structs = Slw(); structs.display()

In [ ]:
strucs = structs.string_list; print(strucs)

Create a folder for each channel inside each condition

In [ ]:
for mod in modalities:
    for chan in channels:
        folder_path_mods_chans = Path(f"../Analysis/{batch_name}/{mod}/{chan}")
        if not folder_path_mods_chans.exists():
            folder_path_mods_chans.mkdir(parents=True)
            
for mod in modalities:
    for chan in channels:
        folder_path_mods_chans = Path(f"../Acquisition/{batch_name}/{mod}/{chan}")
        if not folder_path_mods_chans.exists():
            folder_path_mods_chans.mkdir(parents=True)        

Two ways:

        A 
        1 - Either read directly a file from a microscope. For instance in LIF files we can have several experiments [E] in a same file, each experiment containing different channels.
        2 - Store (temporarily and separately) each file E.
        3 - From the stored files, separe them in channels and store them as different files. We will respect a naming convention.
        4 - Erase the files [E]

        B 
        1 - Provide directly the names of the files path in a list. The most important part in this version of the software, is that each file name has a unique name / prefix
        for instance XPO-6Oligo_1_pos0, that can be used as an unique identifier. We add a subfix '_FISH' (but can be _FISH1, _FISH2) or '_DAPI' depending on the channel.   

# A:  


if needed skip this step go to B.

In [ ]:
path_folder = fp.select_file(initialdir=None)

for lif files:

In [ ]:
images_names, image_list = fp.read_lif_many_images(path_folder) # since it is a lif file, for nd2 files see and adapt method read_convert_np

In [ ]:
import readlif as readlif
from readlif.reader import LifFile
file = LifFile(path_folder)
n_images = len(file.image_list)

print(file)

In [ ]:
len(file.image_list)

In [ ]:
np.shape(image_list[2])

In [ ]:
folder_path_batch = Path(f"../Analysis/{batch_name}/temp_file")
if not folder_path_batch.exists():
    folder_path_batch.mkdir(parents=True)

In [ ]:
path_names = []
for ind, image in enumerate(image_list):
    filename = Path(f"../Analysis/{batch_name}/temp_file") / Path(images_names[ind]+'.tif')
    io.imsave(filename, image, imagej=True)
    path_names.append(filename)

### Manual intervention required: change the channel numbers (for instance channel_dapi ) and verify that the channel corresponds to what you are looking for. 

In [ ]:
print(np.shape(image_list[0])) # dimensions of the images (#channels, # Z, #Y, #X)
channel_dapi = 0               # verify which channels is which 
#channel_fish = 1
fig, ax = plt.subplots(figsize=(1,1))  
ax.imshow(np.max(image_list[0][channel_dapi,...], axis = 0)) # for 3D images
ax.set_axis_off();

In [ ]:
channel_dapi = 0     # this line is useful to separate the channels
#channel_fish = 1
print(channels)
channels_num  = [channel_dapi]#, channel_fish]
print('corresponding channels :',  channels_num)

##### For each condition: manually affect the experiments to each condition:
   Among the list of image names, choose the ones that must be affected to each condition

In [ ]:
print(images_names)
list_same_cond = [images_names[0], images_names[1], images_names[3]]   # Choose the images for a given condition.
indexes_ims    = [0, 1, 3]                                             # copy the indexes of the images 
name_cond      = 'LNP'                                                 # the name of the condition/modalities must be among the modalities listed just before. 

In [ ]:
for ind_image, name in enumerate(list_same_cond):                      # This cell separates the files in channels and stores them in the corresponding condition / channel folder
    folder_dirs = []
    for ind, chan in enumerate(channels_num):
        im_c      = image_list[indexes_ims[ind]][chan,...]
        file_path = str(Path(f"../Acquisition/{batch_name}/{name_cond}")/ Path(channels[ind]) / Path(name + f'_{channels[ind]}.tif'))
        io.imsave(file_path, im_c)
        folder_dirs.append(file_path)
        
    exec(f"BATCH_{name_cond}_{channels[ind]} = folder_dirs", globals())  # create the uppercased variables  (condition, channel) containing the list of all the associated files
                                                                         # for instance BATCH_LNP_DAPI        

Once all the channels of each condition were affected in a specific folder as a separate .tif file, erase the /temp_file folder

In [ ]:
if os.path.exists(Path(f"../Analysis/{batch_name}/temp_file")):
    shutil.rmtree(Path(f"../Analysis/{batch_name}/temp_file"))  # Remove the folder and all its contents

# B    
Provide directly the files names with the good names on them. If the files are already separated in conditions and in different channels (one tiff per channel).
Just reference them using the following synthax: BATCH_{condition}_{channel}, both conditions and channel in uppercase.

Warning : naming convention. We will assume in the rest of the pipeline, that the files between the name channels have similar names, excepting for the last suffix
which is for instance _DAPI or _FISH or _FISH0 or _FISH1 (you can decide on how to call them)

In [ ]:
channels_num   = [0, 1]

BATCH_KRAKEN_DAPI = [Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s24_DAPI.tif"),
                ]


                     
BATCH_KRAKEN_FISH = [Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r1_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r2_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r3_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r4_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r5_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r6_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r7_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_D_R1_r8_s24_FISH.tif"),
                    ]   



BATCH_KRAKEN_FISH_old = [Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r2_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r3_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r4_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r5_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r6_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r7_s24_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s00_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s01_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s02_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s03_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s04_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s05_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s06_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s07_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s08_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s09_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s10_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s11_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s12_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s13_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s14_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s15_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s16_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s17_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s18_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s19_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s20_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s21_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s22_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s23_FISH.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan3_D_R1_r8_s24_FISH.tif"),]



BATCH_KRAKEN_DAPI_old = [Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T2_D_R1_star/T2_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r1_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r2_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r3_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r4_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r5_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r6_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r7_s24_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s00_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s01_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s02_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s03_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s04_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s05_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s06_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s07_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s08_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s09_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s10_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s11_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s12_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s13_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s14_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s15_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s16_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s17_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s18_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s19_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s20_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s21_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s22_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s23_DAPI.tif"),
                    Path("/media/PBI_big_data/rna-imaging/data/smFISH-optimization/KRAKEN/2025-08-25_kraken_autofish/T3_D_R1_star/T3_D_R1_grouped/2025-08-25_Kraken-Papier3_TileScan2_D_R1_r8_s24_DAPI.tif"),]



for u in BATCH_KRAKEN_FISH:        # check those files exist:
     print(os.path.isfile(u))     
print(' \n')    
for u in BATCH_KRAKEN_FISH:
     print(os.path.isfile(u))

Copy the folders locally to ../Acquisition/{batch_name}/{modality}/{channel}:

In [ ]:
# for mod in modalities:    
#     for chan in channels:
#         file_path = eval(f'BATCH_{mod}_{chan}')
#         for file in file_path:
#             im_c = io.imread(file)
#             name = Path(file).name
#             file_path = str(Path(f"../Acquisition/{batch_name}/{mod}")/ Path(chan) / name)
#             io.imsave(file_path, im_c)

# End of A or B

#### Finally execute these last cells to collect the constants.
#### The convention is to write the constants that we want to pass by to the other notebooks in uppercase at the end of the notebook.
#### This will create a json file that will be used to communicate between notebooks used in a same batch of experiments.

In [ ]:
MODALITIES   = modalities
CHANNELS     = channels
BATCH_NAME   = batch_name
CHANNELS_NUM = channels_num
COLORS       = colors
STRUCTURES   = strucs

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, BATCH_NAME, folder_path = Path(f"../Analysis/{BATCH_NAME}"))